In [ ]:
import boto3
import pandas as pd
import psycopg2

# AWS Configuration
aws_access_key = 'YOUR_AWS_ACCESS_KEY'
aws_secret_key = 'YOUR_AWS_SECRET_KEY'
region_name = 'us-east-1'
redshift_endpoint = 'redshift-cluster.endpoint.amazonaws.com'
redshift_user = 'username'
redshift_password = 'password'
redshift_db = 'dev'

# Initialize AWS Glue client
glue_client = boto3.client('glue',
                           aws_access_key_id=aws_access_key,
                           aws_secret_access_key=aws_secret_key,
                           region_name=region_name)

# Function to trigger Glue ETL job
def trigger_glue_job(job_name, args={}):
    response = glue_client.start_job_run(JobName=job_name, Arguments=args)
    return response['JobRunId']

# Function to load data into Redshift
def load_data_to_redshift(df, table_name):
    conn = psycopg2.connect(
        dbname=redshift_db,
        user=redshift_user,
        password=redshift_password,
        host=redshift_endpoint,
        port='5439'
    )
    cursor = conn.cursor()
    
    # Create table if not exists
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        ds DATE,
        y FLOAT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    
    # Insert data
    for index, row in df.iterrows():
        insert_query = f"INSERT INTO {table_name} (ds, y) VALUES ('{row['ds']}', {row['y']});"
        cursor.execute(insert_query)
    conn.commit()
    cursor.close()
    conn.close()
    print("Data loaded into Redshift.")

# Example Usage
if __name__ == "__main__":
    # Trigger AWS Glue ETL job to process raw data
    glue_job_name = 'business_sales_etl'
    job_run_id = trigger_glue_job(glue_job_name)
    print(f"Triggered Glue job with Run ID: {job_run_id}")
    
    # Load processed data into Redshift
    processed_data = pd.read_csv('processed_business_sales.csv')  # Assume Glue job outputs this
    load_data_to_redshift(processed_data, 'business_sales')
